<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/Language/10_AutoGPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# パッケージのインストール
!git clone https://github.com/PanQiWei/AutoGPTQ
%cd AutoGPTQ
!git reset --hard e7c8fbeafa9a560f038b389b2b90b0360c904807

Cloning into 'AutoGPTQ'...
remote: Enumerating objects: 3965, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 3965 (delta 71), reused 80 (delta 44), pack-reused 3848
Receiving objects: 100% (3965/3965), 7.93 MiB | 10.73 MiB/s, done.
Resolving deltas: 100% (2538/2538), done.
/content/AutoGPTQ
HEAD is now at e7c8fbe Only make_quant on inside_layer_modules. (#479)


In [2]:
!pip install .
!pip install transformers==4.35.2 accelerate==0.25.0 datasets==2.15.0 peft==0.7.0 gekko

Processing /content/AutoGPTQ
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.8 MB/s eta

In [3]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

# トークナイザーとモデルの準備
model_name_or_path = "TheBloke/vicuna-7B-v1.5-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True
)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
    use_safetensors=True,
    trust_remote_code=False,
    device="cuda:0",
    use_triton=False,
    quantize_config=None
)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

1. You disabled CUDA extensions compilation by setting BUILD_CUDA_EXT=0 when install auto_gptq from source.
2. You are using pytorch without CUDA support.
3. CUDA and nvcc are not installed in your device.


quantize_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

In [4]:
prompt=f'''USER: 機械学習って何？
ASSISTANT: '''

# 推論の実行
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()

In [5]:
%%time
output = model.generate(inputs=input_ids, temperature=0.1, max_new_tokens=512)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


CPU times: user 4min 26s, sys: 2min 15s, total: 6min 41s
Wall time: 6min 47s


In [6]:
print(tokenizer.decode(output[0]))

<s> USER: 機械学習って何？
ASSISTANT: 機械学习（英文：Machine Learning）是一种人工智能技术，它利用计算机自动从数据中学习，以便对新的数据进行预测、分类、推荐等任务。机器学习的目标是让计算机能够从数据中自动提取知识，而不需要人工指导。

机器学习的基本思想是通过训练算法来让计算机从大量数据中学习，以便在未知数据上进行预测。这种训练过程通常涉及以下步骤：

1. 数据收集：收集大量与目标相关的数据。
2. 数据预处理：对收集的数据进行清洗、转换等处理，以便于后续的分析。
3. 特征工程：从原始数据中提取有用的特征，以便于模型的训练。
4. 模型选择：选择合适的机器学习算法来训练模型。
5. 模型训练：使用训练数据训练模型，以便于在未知数据上进行预测。
6. 模型评估：评估训练好的模型，以确定其性能如何。
7. 模型优化：根据评估结果对模型进行优化，以提高其性能。

机器学习在很多领域都有应用，例如自然语言处理、图像
